# Upload files from google drive to here. Only do this once

In [0]:
# kill VM
# !kill -9 -1
!ls

In [0]:
%%capture
# # install pyDrive
!pip install PyDrive


# Authorize google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# %%capture
# code for download file from google drive
fileId = drive.CreateFile({'id': 'INSERT ID OF THE DATA FOLDER'})
print(fileId['title'])  # folder_data.zip
fileId.GetContentFile('tire_detection_v2_colab.zip')  # Save Drive file as a local file

!unzip -o -qq tire_detection_v2_colab.zip -d ./

# remove this useless file
!rm -rf __MACOSX

# rename the file for convenience
!mv tire_detection_v2_colab tire_detection

# Preprocessing and path requirements

In [0]:
%%capture
# install dependencies
!pip install -r requirements.txt

# install the protobuf compiler
!apt-get install protobuf-compiler

In [0]:
import numpy as np
import tensorflow as tf
import os
import sys
from IPython.display import Image

In [0]:
# change the current directory
os.chdir("/content/tire_detection/")

# append slim folder to the python path
sys.path.append('/content/tire_detection/slim')

In [0]:
%%capture
# compile files in the object detection git
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
# %%capture
# # create the record files from images and annotations
# %run create_tf_record.py --data_dir=pwd --output_dir= pwd

# Select a pretrained model and download it.

In [0]:
models = ['ssdlite_mobilenet_v2_coco_2018_05_09', 'faster_rcnn_resnet50_coco_2018_01_28', 
          'faster_rcnn_inception_v2_coco_2018_01_28', 'ssd_mobilenet_v2_coco_2018_03_29', 
          'ssd_inception_v2_coco_2018_01_28']

model_name = models[4]

In [0]:
%%capture
# if the model doesn't exist download it
if not os.path.exists(os.getcwd() + '/pretrained_models/%s' % model_name):
  # model dir
  model_dir = 'http://download.tensorflow.org/models/object_detection/%s.tar.gz' % model_name

  # import the model
  !wget "$model_dir" -P "./pretrained_models/"

  # extract the model
  !tar -xzf ./pretrained_models/"$model_name".tar.gz -C ./pretrained_models

  # # remove the tar file
  !rm ./pretrained_models/"$model_name".tar.gz

In [0]:
# check that the pretrained model is in the right directory
ls pretrained_models/$model_name

# Train the model using our wheel data

**IF YOU HAVE TRAINING POINTS, INPUT BELOW**

In [0]:
ls

In [0]:
# IF YOU HAVE ANY CHECKPOINT, PUT FILE ID. Else, put None

check_id = '1t2ambHAy53_R-crM0KRktf7YOo6X5jIp'  # None if there are no checkpoints

fileId = drive.CreateFile({'id': check_id}) # MODIFY THE FILE ID
try:
  fileId.GetContentFile("%s_train.tar.gz" % model_name)
  
  # unzip the file
  !tar -xf "$model_name"_train.tar.gz
  
  # replace the checkpoint files in train directory
  !cp -r ./"$model_name" ./train
  
  # remove the train file
  !rm -rf $model_name
  
  # remove the tar file
  !rm "$model_name"_train.tar.gz

except:
  print('Couldnt find any checkpoints')
  pass

In [0]:
# Make sure all the directories are good and train
ls ./train/$model_name/

In [0]:
# train
pipeline_config_path="pipeline_config_files/%s.config" % model_name
train_path="train/%s/" % model_name
%run object_detection/train.py \
  --logtostderr \
  --pipeline_config_path=$pipeline_config_path \
  --train_dir=$train_path


/content/tire_detection/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 87, in catch_config

Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Use the `axis` argument instead
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0


Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession


Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession


INFO:tensorflow:Restoring parameters from train/ssd_inception_v2_coco_2018_01_28/model.ckpt-10267


...

INFO:tensorflow:global step 15251: loss = 2.4765 (1.547 sec/step)


INFO:tensorflow:global step 15252: loss = 4.1493 (1.546 sec/step)


INFO:tensorflow:global step 15252: loss = 4.1493 (1.546 sec/step)


INFO:tensorflow:global step 15253: loss = 2.6045 (1.541 sec/step)


INFO:tensorflow:global step 15253: loss = 2.6045 (1.541 sec/step)


# Export the frozen graph from the trained model

In [0]:
import os
import sys

# change the current directory
os.chdir("/content/tire_detection/")

# append slim folder to the python path
sys.path.append('/content/tire_detection/slim')

In [0]:
models = ['ssdlite_mobilenet_v2_coco_2018_05_09', 'faster_rcnn_resnet50_coco_2018_01_28', 
          'faster_rcnn_inception_v2_coco_2018_01_28', 'ssd_mobilenet_v2_coco_2018_03_29', 
          'ssd_inception_v2_coco_2018_01_28']

model_name = models[0]

In [0]:
# check which checkpoint to import
ls train/$model_name

In [0]:
 # make a directory for the model if it doesn't exist
cwd = os.getcwd()
if not os.path.exists(cwd + '/frozen_graph/%s/' % model_name):
 os.mkdir(cwd + '/frozen_graph/%s' % model_name)

# paths
pipeline_config_path="pipeline_config_files/%s.config" % model_name
checkpoint_path="train/%s/model.ckpt-10589" % model_name
output_path = "frozen_graph/%s" % model_name

# Export the frozen graph model. Will obtain frozen_inference_graph.pb
%run object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path=$pipeline_config_path \
--trained_checkpoint_prefix=$checkpoint_path \
--output_directory=$output_path


# Run on test images

In [0]:
if not os.path.exists(cwd + '/results/%s/img' % model_name):
  os.mkdir(cwd + '/results/%s/img' % model_name)

# paths for test image and final output
input_path="./test_images/img"
output_path="./results/$model_name/img"
label_path="./annotations/label_map.pbtxt"
frozen_graph_path="./frozen_graph/$model_name/frozen_inference_graph.pb"

In [0]:
# run on test set
%run object_detection/inference.py \
  --input_dir=$input_path \
  --output_dir=$output_path \
  --label_map=$label_path \
  --frozen_graph=$frozen_graph_path \
  --num_output_classes=1

In [0]:
ls object_detection/inference/

# Display results of test images

In [0]:
from IPython.display import Image
cwd = os.getcwd()
result_dir = cwd + '/results/$img/'
Image(result_dir + 'VID_20171024_110744_2_33_0.jpg')

# Run evaluation with metrics: WORK IN PROGRESS

In [0]:
#https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_locally.md
# From the tensorflow/models/ directory
python object_detection/eval.py \
    --logtostderr \
    --pipeline_config_path=pipeline.config\
    --checkpoint_dir=train/ \
    --eval_dir=results/eval

In [0]:
# does this work? 
tensorboard --logdir=${PATH_TO_MODEL_DIRECTORY}

# Save checkpoint files from colab to google drive

In [0]:
ls /content

In [0]:
# %%capture
# Zip the contents of the frozen graph, results and train directory
# !tar -zcf frozen_graph.tar.gz ./frozen_graph && mv ./frozen_graph.tar.gz /content
# !tar -zcvf results.tar.gz ./results && mv ./results.tar.g3z /content

# tar and save the checkpoints
# !tar -zcvf "$model_name"_train.tar.gz ./train/"$model_name"
!tar -C ./train -zcf "$model_name"_train.tar.gz "$model_name"
!mv ./"$model_name"_train.tar.gz /content

Print the file id of all the folders to find the folder to put the results in.

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

Upload the files to google drive

In [0]:
# change directory
cd

In [0]:
# file = drive.CreateFile({'parents':[{u'id': None}]}) 
# file.SetContentFile("frozen_graph.tar.gz")
# file.Upload()

# file = drive.CreateFile({'parents':[{u'id': None}]}) 
# file.SetContentFile("results.tar.gz")
# file.Upload()

file = drive.CreateFile({'parents':[{u'id': '15cN51ilhw5nMa0CgmlcjNPxPtBALE0Dh'}]}) 
file.SetContentFile("%s_train.tar.gz" % model_name)
file.Upload()

In [0]:
ls